# 如何保存和加载 LangChain 对象

LangChain 类实现了标准的序列化方法。使用这些方法序列化 LangChain 对象具有一些优势：

- 秘密信息（如 API 密钥）与其他参数分开，并在反序列化时可以加载回对象；
- 反序列化在不同包版本之间保持兼容，因此用一个版本的 LangChain 序列化的对象可以使用另一个版本正确地反序列化。

要使用此系统保存和加载 LangChain 对象，请使用 `langchain-core` 的 [load 模块](https://python.langchain.com/api_reference/core/load.html) 中的 `dumpd`、`dumps`、`load` 和 `loads` 函数。这些函数支持 JSON 和可 JSON 序列化的对象。

所有继承自 [Serializable](https://python.langchain.com/api_reference/core/load/langchain_core.load.serializable.Serializable.html) 的 LangChain 对象都是可 JSON 序列化的。例如，像 [消息](https://python.langchain.com/api_reference//python/core_api_reference.html#module-langchain_core.messages)、[文档对象](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)（例如从 [检索器](/docs/concepts/retrievers) 返回的对象）以及大多数 [Runnables](/docs/concepts/lcel)，例如使用 LangChain Expression Language 实现的聊天模型、检索器和[链条](/docs/how_to/sequence)。

下面我们将通过一个简单的 [LLM 链](/docs/tutorials/llm_chain) 的示例进行介绍。

:::caution

使用 `load` 和 `loads` 进行反序列化可以实例化任何可序列化的 LangChain 对象。请仅对受信任的输入使用此功能！

反序列化是一项 Beta 功能，可能会发生变更。
:::

In [12]:
from langchain_core.load import dumpd, dumps, load, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Translate the following into {language}:"),
        ("user", "{text}"),
    ],
)

llm = ChatOpenAI(model="gpt-4o-mini", api_key="llm-api-key")

chain = prompt | llm

## 保存对象

### 转换为JSON

In [2]:
string_representation = dumps(chain, pretty=True)
print(string_representation[:500])

{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "schema",
    "runnable",
    "RunnableSequence"
  ],
  "kwargs": {
    "first": {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "prompts",
        "chat",
        "ChatPromptTemplate"
      ],
      "kwargs": {
        "input_variables": [
          "language",
          "text"
        ],
        "messages": [
          {
            "lc": 1,
            "type": "constructor",
         


### 转换为 JSON 可序列化的 Python 字典

In [3]:
dict_representation = dumpd(chain)

print(type(dict_representation))

<class 'dict'>


### 写入磁盘

In [4]:
import json

with open("/tmp/chain.json", "w") as fp:
    json.dump(string_representation, fp)

请注意，API 密钥已从序列化表示中隐藏。被视为秘密的参数由 LangChain 对象的 `.lc_secrets` 属性指定：

In [5]:
chain.last.lc_secrets

{'openai_api_key': 'OPENAI_API_KEY'}

## 加载对象

在 `load` 和 `loads` 中指定 `secrets_map` 会将对应的 secrets 加载到反序列化的 LangChain 对象上。

### 从字符串加载

In [7]:
chain = loads(string_representation, secrets_map={"OPENAI_API_KEY": "llm-api-key"})

### 从字典

In [9]:
chain = load(dict_representation, secrets_map={"OPENAI_API_KEY": "llm-api-key"})

### 从磁盘

In [10]:
with open("/tmp/chain.json", "r") as fp:
    chain = loads(json.load(fp), secrets_map={"OPENAI_API_KEY": "llm-api-key"})

请注意，我们恢复了指南开头指定的 API 密钥：

In [11]:
chain.last.openai_api_key.get_secret_value()

'llm-api-key'